In [1]:
import pandas as pd
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [6]:
pickle_in1 = open("../Twitter_bot_detection_713/data/X_train_embed.pickle","rb")
X_train_embed = pickle.load(pickle_in1)
pickle_in2 = open("../Twitter_bot_detection_713/data/X_test_embed.pickle","rb")
X_test_embed = pickle.load(pickle_in2)
pickle_in3 = open("../Twitter_bot_detection_713/data/y_train.pickle","rb")
y_train = pickle.load(pickle_in3)
pickle_in4 = open("../Twitter_bot_detection_713/data/y_test.pickle","rb")
y_test = pickle.load(pickle_in4)

In [9]:
X_train_embed[0:100]

519816     [[0.31553, 0.53765, 0.10177, 0.032553, 0.00379...
893128     [[-0.17478, -0.10639, -0.086791, 0.10731, -0.4...
256624     [[0.31553, 0.53765, 0.10177, 0.032553, 0.00379...
2010377    [[0.31553, 0.53765, 0.10177, 0.032553, 0.00379...
624038     [[0.31553, 0.53765, 0.10177, 0.032553, 0.00379...
                                 ...                        
30024      [[-0.4509, 0.077681, -0.058347, 0.2859, -0.369...
637277     [[-0.48187, -0.29707, -0.77723, -0.2486, -0.04...
1944909    [[0.31553, 0.53765, 0.10177, 0.032553, 0.00379...
2209087    [[0.27913, -0.14389, -0.0036183, -0.4946, -0.1...
359284     [[0.073707, 0.43199, 0.24253, 0.015394, 0.0181...
Name: clean_text, Length: 100, dtype: object

In [3]:
# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=60)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=60)

In [8]:
X_train_pad[0].shape

(60, 200)

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [15]:
def initialize_model():
    
    model = Sequential()
    
    model.add(layers.Masking(mask_value=0.0, input_shape=(60,200)))
    
    model.add(layers.Conv1D(20, kernel_size=3))
    
    model.add(layers.Conv1D(20, kernel_size=3))
    
    model.add(layers.Conv1D(20, kernel_size=3))
    
    model.add(layers.Conv1D(20, kernel_size=3))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(20, activation='relu'))
    
    model.add(layers.Dense(10, activation='relu'))
    
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                 optimizer='adam',
                  metrics=['accuracy', 'Precision','Recall']
                 )
    return model

In [16]:
model = initialize_model()

es = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(X_train_pad, y_train,
                   validation_split=0.2,
                   batch_size=64,
                   epochs=50,
                   callbacks=[es])

Epoch 1/50
16130/16130 [==============================] - 495s 31ms/step - loss: 0.3550 - accuracy: 0.8720 - precision: 0.6629 - recall: 0.1012 - val_loss: 0.3468 - val_accuracy: 0.8748 - val_precision: 0.7898 - val_recall: 0.1001
Epoch 2/50
16130/16130 [==============================] - 571s 35ms/step - loss: 0.3441 - accuracy: 0.8764 - precision: 0.7157 - recall: 0.1362 - val_loss: 0.3417 - val_accuracy: 0.8771 - val_precision: 0.7093 - val_recall: 0.1532
Epoch 3/50
16130/16130 [==============================] - 909s 56ms/step - loss: 0.3392 - accuracy: 0.8785 - precision: 0.7307 - recall: 0.1543 - val_loss: 0.3406 - val_accuracy: 0.8787 - val_precision: 0.6825 - val_recall: 0.1920
Epoch 4/50
16130/16130 [==============================] - 674s 42ms/step - loss: 0.3356 - accuracy: 0.8803 - precision: 0.7427 - recall: 0.1698 - val_loss: 0.3389 - val_accuracy: 0.8787 - val_precision: 0.8053 - val_recall: 0.1349
Epoch 5/50
16130/16130 [==============================] - 569s 35ms/step - l

KeyboardInterrupt: 

In [10]:
from tensorflow.keras.layers import Bidirectional, LSTM

def initialize_model2():
    
    model = Sequential()
    
    model.add(layers.Masking(mask_value=0.0, input_shape=(60,200)))
    
    model.add(Bidirectional(LSTM(20, return_sequences=True)))
   
    model.add(Bidirectional(LSTM(20, return_sequences=True)))

    model.add(Bidirectional(LSTM(20, return_sequences=False)))
    
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                 optimizer='rmsprop',
                  metrics=['accuracy', 'Precision','Recall']
                 )
    return model

In [11]:
rnn = initialize_model2()
rnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_2 (Masking)          (None, 60, 200)           0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 60, 40)            35360     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 60, 40)            9760      
_________________________________________________________________
bidirectional_8 (Bidirection (None, 40)                9760      
_________________________________________________________________
dense (Dense)                (None, 1)                 41        
Total params: 54,921
Trainable params: 54,921
Non-trainable params: 0
_________________________________________________________________


In [14]:
es = EarlyStopping(patience=5, restore_best_weights=True)

history2 = rnn.fit(X_train_pad, y_train, 
          epochs=30, 
          batch_size=128,
          validation_split=0.3,
          callbacks=[es])

Epoch 1/30
7057/7057 [==============================] - 627s 89ms/step - loss: 0.3284 - accuracy: 0.8823 - precision: 0.7363 - recall: 0.1985 - val_loss: 0.3159 - val_accuracy: 0.8890 - val_precision: 0.7339 - val_recall: 0.2734
Epoch 2/30
7057/7057 [==============================] - 637s 90ms/step - loss: 0.3061 - accuracy: 0.8937 - precision: 0.7932 - recall: 0.2868 - val_loss: 0.3044 - val_accuracy: 0.8952 - val_precision: 0.7697 - val_recall: 0.3144
Epoch 3/30
7057/7057 [==============================] - 1135s 161ms/step - loss: 0.2951 - accuracy: 0.8987 - precision: 0.8179 - recall: 0.3201 - val_loss: 0.2993 - val_accuracy: 0.8978 - val_precision: 0.8559 - val_recall: 0.2884
Epoch 4/30
7057/7057 [==============================] - 637s 90ms/step - loss: 0.2879 - accuracy: 0.9023 - precision: 0.8366 - recall: 0.3429 - val_loss: 0.2974 - val_accuracy: 0.8988 - val_precision: 0.7946 - val_recall: 0.3334
Epoch 5/30
7057/7057 [==============================] - 629s 89ms/step - loss: 0.2

In [15]:
rnn.evaluate(X_test_pad, y_test)

10082/10082 [==============================] - 114s 11ms/step - loss: 0.2930 - accuracy: 0.9021 - precision: 0.8251 - recall: 0.3466


[0.29300349950790405,
 0.9020570516586304,
 0.825146496295929,
 0.34662187099456787]

In [16]:
def initialize_model3():
    
    model = Sequential()
    
    model.add(layers.Masking(mask_value=0.0, input_shape=(60,200)))
    
    model.add(Bidirectional(LSTM(20, return_sequences=True)))
   
    model.add(Bidirectional(LSTM(20, return_sequences=True)))

    model.add(Bidirectional(LSTM(20, return_sequences=False)))
    
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                 optimizer='adam',
                  metrics=['accuracy', 'Precision','Recall']
                 )
    return model

In [17]:
rnn2 = initialize_model3()

history3 = rnn2.fit(X_train_pad, y_train, 
          epochs=30, 
          batch_size=128,
          validation_split=0.3,
          callbacks=[es])

Epoch 1/30
7057/7057 [==============================] - 637s 89ms/step - loss: 0.3347 - accuracy: 0.8799 - precision: 0.7255 - recall: 0.1770 - val_loss: 0.3157 - val_accuracy: 0.8884 - val_precision: 0.7304 - val_recall: 0.2684
Epoch 2/30
7057/7057 [==============================] - 625s 89ms/step - loss: 0.3072 - accuracy: 0.8929 - precision: 0.7866 - recall: 0.2829 - val_loss: 0.3032 - val_accuracy: 0.8955 - val_precision: 0.8343 - val_recall: 0.2775
Epoch 3/30
7057/7057 [==============================] - 675s 96ms/step - loss: 0.2951 - accuracy: 0.8989 - precision: 0.8194 - recall: 0.3214 - val_loss: 0.2987 - val_accuracy: 0.8980 - val_precision: 0.8501 - val_recall: 0.2929
Epoch 4/30
7057/7057 [==============================] - 687s 97ms/step - loss: 0.2874 - accuracy: 0.9025 - precision: 0.8369 - recall: 0.3447 - val_loss: 0.2974 - val_accuracy: 0.8986 - val_precision: 0.7695 - val_recall: 0.3504
Epoch 5/30
  38/7057 [..............................] - ETA: 10:36 - loss: 0.2633 - 

KeyboardInterrupt: 